In [1]:
class Student(object):

    def __init__(self, id, first_name, last_name, email) -> None:
        self.id = id
        self.first_name = first_name
        self.last_name = last_name
        self.email = email

class Team(object):

    def __init__(self, members) -> None:
        self.members = members

    def add_member(self, member):
        if self.members == None:
            self.members = []
        self.members.append(member)
    


In [2]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import sys

# Export path: change before execution:
EXPORT_PATH = './../data/teams_ML_TP_00.txt'

data = pd.read_csv("./../data/student_list.csv")
# Create a DataFrame of students
data = {
    'id': data.id,
    'first_name': data.first_name,
    'last_name': data.last_name,
    'email': data.email
}


current_team = None
teams = []
current_team = set()

df = pd.DataFrame.from_dict(data)

# Create checkboxes for each student
checkboxes = {}
for i, row in df.iterrows():
    checkboxes[row['id']] = widgets.Checkbox(description=f"{row['id']}: {row['first_name']} {row['last_name']}")

# Display checkboxes in a grid layout
checkbox_grid = widgets.GridBox(list(checkboxes.values()), layout=widgets.Layout(grid_template_columns="repeat(3, 300px)"))
display(checkbox_grid)


# Button for validation
validate_button = widgets.Button(description='Create Team')
show_teams_button = widgets.Button(description='Show teams')
show_current_team_button = widgets.Button(description='Show current team')
export_teams_button = widgets.Button(description='Export teams')

# Display button
display(validate_button)
display(show_current_team_button)
display(show_teams_button)
display(export_teams_button)


# Function to handle checkbox changes
def handle_checkbox_change(change):
    for id_, checkbox in checkboxes.items():
        if checkbox == change['owner']:
            # get the current selected or deselected student
            current_record = df.where(df.id == id_).dropna().iloc[0].to_dict()
            current_id = current_record['id']
            
            # print(current_record)

            if change['new']:
                current_team.add(current_id)
            
            else:
                current_team.remove(current_id)

# Attach the same handler to all checkboxes
for checkbox in checkboxes.values():
    checkbox.observe(handle_checkbox_change, names='value')

# Function to handle button click
def handle_create_team_button_click(team_members):
    print('the current team is: {}'.format(team_members))
    # validation phase: make sure that none of the current team members were used in other teams
    if len(team_members) == 0:
        print('Please select at least one member to create a team')
        return
    
    for member in team_members:
        for t in teams:
            if t.__contains__(member):
                print('{} is already used in the team {}'.format(member, t))
                return
    
    # add the current team to the list of teams
    teams.append(team_members.copy())
    
    for key, checkbox in checkboxes.items():
        checkbox.value = False
    current_team = set()


def handle_show_teams_button_click(b):
    print('showing all teams:')
    for t in teams:
        print(t)

    print('---------------------------------------------')

def handle_export_teams_button_click(teams, enable_check_found=False):
    # validation phase:
    # make sure all teams are included
    for stdid in df.id:
        found = False
        for t in teams: 
            if t.__contains__(stdid):
                found = True
                break
        if not found and enable_check_found:
            print('student with id: {} not found'.format(stdid))
            return
    
    print('All students are found')

    # export phase
    with open(EXPORT_PATH, 'w') as f:
        for t in teams:
            team_line = str.join(',', list(t))
            f.write(team_line)
            f.write('\n')
        f.close()


def hand_show_current_team_button_click(b):
    print('the current team:')
    for id in current_team:
        s = df[df.id == id].iloc[0].to_dict()
        print(
            "{}: {} {}".format(id, s['first_name'], s['last_name'])
        )

    print('-------------------------------------')

# Attach the handler to the button
validate_button.on_click(lambda x: handle_create_team_button_click(current_team))
show_teams_button.on_click(handle_show_teams_button_click)
show_current_team_button.on_click(hand_show_current_team_button_click)
export_teams_button.on_click(lambda x: handle_export_teams_button_click(teams))

GridBox(children=(Checkbox(value=False, description='20/0003: HATEM MOHAMED ABDELMOUMEN'), Checkbox(value=Fals…

Button(description='Create Team', style=ButtonStyle())

Button(description='Show current team', style=ButtonStyle())

Button(description='Show teams', style=ButtonStyle())

Button(description='Export teams', style=ButtonStyle())

the current team is: {'20/0025', '20/0050'}
the current team:
-------------------------------------
showing all teams:
{'20/0025', '20/0050'}
---------------------------------------------
the current team is: {'20/0280', '20/0278'}
showing all teams:
{'20/0025', '20/0050'}
{'20/0280', '20/0278'}
---------------------------------------------
the current team is: {'20/0025', '20/0050'}
20/0025 is already used in the team {'20/0025', '20/0050'}
showing all teams:
{'20/0025', '20/0050'}
{'20/0280', '20/0278'}
---------------------------------------------
the current team:
20/0025: AHMED RAYANE KEBIR
20/0050: MAROUANE BENBETKA
-------------------------------------
the current team is: {'20/0025', '20/0050'}
20/0025 is already used in the team {'20/0025', '20/0050'}
All students are found


hellanother